In [9]:
#импортируем нужные библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('MachineLearning/poker-hand-testing.data.csv', sep = ',')

In [10]:
#Смотрим на датасет
df.head()

,1,1.1,1.2,13,2,4,2.1,3,1.3,12,0
0,3,12,3,2,3,11,4,5,2,5,1
1,1,9,4,6,1,4,3,2,3,9,1
2,1,4,3,13,2,13,2,1,3,6,1
3,3,10,2,7,1,2,2,11,4,9,0
4,1,3,4,5,3,4,1,12,4,6,0


In [11]:
'''
Color — масть карты: (1-4) = {черви, пики, буби, крести}  
Rank — ранг карты: (1-12) = {туз, 2, 3, ... дама, король}

HandName - название руки:
0: Мусор 
1: Одна пара;  
2: Две пары;  
3: Тройка;  
4: Стрейт;  
5: Флэш;  
6: Фулл Хаус;  
7: Каре;  
8: Стрейт флэш;  
9: Флэш рояль;  
'''
#Переименовываем столбцы: 
df.columns = ['Color1', 'Rank1', 'Color2', 'Rank2', 
              'Color3', 'Rank3', 'Color4', 'Rank4', 'Color5', 'Rank5',
              'HandName']
df.head()

,Color1,Rank1,Color2,Rank2,Color3,Rank3,Color4,Rank4,Color5,Rank5,HandName
0,3,12,3,2,3,11,4,5,2,5,1
1,1,9,4,6,1,4,3,2,3,9,1
2,1,4,3,13,2,13,2,1,3,6,1
3,3,10,2,7,1,2,2,11,4,9,0
4,1,3,4,5,3,4,1,12,4,6,0
